In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader


In [2]:
# 准备数据集
class TitanicDataset(Dataset):
    def __init__(self, filepath):
        xy = pd.read_csv(filepath)
        # xy.shape（）可以得到xy的行列数
        self.len = xy.shape[0]
        # 选取相关的数据特征
        feature = ["Pclass", "Sex", "SibSp", "Parch", "Fare"]
        # np.array()将数据转换成矩阵，方便进行接下来的计算
        self.x_data = torch.from_numpy(np.array(pd.get_dummies(xy[feature])))
        self.y_data = torch.from_numpy(np.array(xy["Survived"]))

    # getitem函数，可以使用索引拿到数据
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # 返回数据的条数/长度
    def __len__(self):
        return self.len


# 实例化自定义类，并传入数据地址
dataset = TitanicDataset('train.csv')
# num_workers是否要进行多线程服务，num_worker=2 就是2个进程并行运行
# 采用Mini-Batch的训练方法
train_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=True, num_workers=0)

In [3]:
# 定义模型
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # 要先对选择的特征进行独热表示计算出维度，而后再选择神经网络开始的维度
        self.linear1 = torch.nn.Linear(6, 3)
        self.linear2 = torch.nn.Linear(3, 1)

        self.sigmoid = torch.nn.Sigmoid()

    # 前馈
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))

        return x

    # 测试函数
    def test(self, x):
        with torch.no_grad():
            x = self.sigmoid(self.linear1(x))
            x = self.sigmoid(self.linear2(x))
            y = []
            # 根据二分法原理，划分y的值
            for i in x:
                if i > 0.5:
                    y.append(1)
                else:
                    y.append(0)
            return y


# 实例化模型
model = Model()

In [6]:
# 定义损失函数
criterion = torch.nn.BCELoss(reduction='mean')
# 定义优化器
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:
# 防止windows系统报错
if __name__ == '__main__':
    # 采用Mini-Batch的方法训练要采用多层嵌套循环
    # 所有数据都跑100遍
    for epoch in range(100):
        # data从train_loader中取出数据（取出的是一个元组数据）：（x，y）
        # enumerate可以获得当前是第几次迭代，内部迭代每一次跑一个Mini-Batch
        for i, data in enumerate(train_loader, 0):
            # inputs获取到data中的x的值，labels获取到data中的y值
            x, y = data
            x = x.float()
            y = y.float()
            y_pred = model(x)
            y_pred = y_pred.squeeze(-1)
            loss = criterion(y_pred, y)
            print(epoch, i, loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

0 0 0.8916522860527039
0 1 0.5024170279502869
0 2 0.8963754773139954
0 3 0.8790242671966553
0 4 0.5055120587348938
0 5 0.8726913332939148
0 6 0.543816089630127
0 7 0.5045838356018066
0 8 0.9243062734603882
0 9 0.5054078102111816
0 10 0.8870915174484253
0 11 0.506180465221405
0 12 0.8705524206161499
0 13 0.865027129650116
0 14 0.5455754995346069
0 15 0.8652417659759521
0 16 0.911963164806366
0 17 0.5122323632240295
0 18 0.8572227358818054
0 19 0.9129972457885742
0 20 0.8631117939949036
0 21 0.5340895652770996
0 22 0.9085810780525208
0 23 0.9040806889533997
0 24 0.5208351612091064
0 25 0.8423786163330078
0 26 0.8715789914131165
0 27 0.5239155292510986
0 28 0.5378594994544983
0 29 0.8672317862510681
0 30 0.8592411875724792
0 31 0.827104389667511
0 32 0.82886803150177
0 33 0.529826819896698
0 34 0.5284368395805359
0 35 0.8285495042800903
0 36 0.5308440327644348
0 37 0.8193405866622925
0 38 0.8135731816291809
0 39 0.5521044135093689
0 40 0.8002427816390991
0 41 0.8351361155509949
0 42 0.784

In [8]:
# 测试
test_data = pd.read_csv('test.csv')
feature = ["Pclass", "Sex", "SibSp", "Parch", "Fare"]
test = torch.from_numpy(np.array(pd.get_dummies(test_data[feature])))
y = model.test(test.float())

# 输出预测结果
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y})
output.to_csv('my_predict.csv', index=False)

In [19]:
data = pd.read_csv("Bitcoin.csv")
all_data = data['Bitcoin'].to_numpy()

In [20]:
all_data

array([  621.65,   609.67,   610.92, ..., 46809.17, 46078.38, 46368.69])

In [21]:
test_data_size = 200

train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]

In [22]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))

In [23]:
print(train_data_normalized[:5])

[[-0.99903083]
 [-0.99945196]
 [-0.99940802]
 [-0.99948184]
 [-0.999427  ]]


In [24]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

In [25]:
train_window = 12


def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L - tw):
        train_seq = input_data[i:i + tw]
        train_label = input_data[i + tw:i + tw + 1]
        inout_seq.append((train_seq, train_label))
    return inout_seq


train_inout_seq = create_inout_sequences(train_data_normalized, train_window)
train_inout_seq[:5]

[(tensor([-0.9990, -0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9994,
          -0.9994, -0.9995, -0.9998, -0.9999]),
  tensor([-1.])),
 (tensor([-0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9994, -0.9994,
          -0.9995, -0.9998, -0.9999, -1.0000]),
  tensor([-0.9997])),
 (tensor([-0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9994, -0.9994, -0.9995,
          -0.9998, -0.9999, -1.0000, -0.9997]),
  tensor([-0.9997])),
 (tensor([-0.9995, -0.9994, -0.9995, -0.9995, -0.9994, -0.9994, -0.9995, -0.9998,
          -0.9999, -1.0000, -0.9997, -0.9997]),
  tensor([-0.9998])),
 (tensor([-0.9994, -0.9995, -0.9995, -0.9994, -0.9994, -0.9995, -0.9998, -0.9999,
          -1.0000, -0.9997, -0.9997, -0.9998]),
  tensor([-0.9996]))]

In [30]:
from torch import nn


class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.lstm = nn.LSTM(input_size, hidden_layer_size)

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]


model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

        #if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   0 loss: 1.05383539
epoch:   0 loss: 0.99736685
epoch:   0 loss: 0.94173592
epoch:   0 loss: 0.88553345
epoch:   0 loss: 0.82762700
epoch:   0 loss: 0.76858008
epoch:   0 loss: 0.70732826
epoch:   0 loss: 0.64256847
epoch:   0 loss: 0.57389367
epoch:   0 loss: 0.50075638
epoch:   0 loss: 0.42188704
epoch:   0 loss: 0.33695009
epoch:   0 loss: 0.24655665
epoch:   0 loss: 0.15369037
epoch:   0 loss: 0.06747914
epoch:   0 loss: 0.00829555
epoch:   0 loss: 0.01366116
epoch:   0 loss: 0.09352594
epoch:   0 loss: 0.13751431
epoch:   0 loss: 0.11513879
epoch:   0 loss: 0.06543724
epoch:   0 loss: 0.02374753
epoch:   0 loss: 0.00311070
epoch:   0 loss: 0.00080207
epoch:   0 loss: 0.00895244
epoch:   0 loss: 0.02050064
epoch:   0 loss: 0.03098523
epoch:   0 loss: 0.03800617
epoch:   0 loss: 0.04075358
epoch:   0 loss: 0.03915724
epoch:   0 loss: 0.03454094
epoch:   0 loss: 0.02750513
epoch:   0 loss: 0.01922115
epoch:   0 loss: 0.01120172
epoch:   0 loss: 0.00453818
epoch:   0 loss: 0.0

In [31]:
fut_pred = 12
test_inputs = train_data_normalized[-train_window:].tolist()
print(test_inputs)

[0.556728720664978, 0.6669414639472961, 0.6478890776634216, 0.6378278732299805, 0.6917944550514221, 0.664778470993042, 0.7072523832321167, 0.8112401962280273, 0.793309211730957, 0.9447616338729858, 0.9477391242980957, 1.0]


In [32]:
model.eval()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

In [33]:
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:]).reshape(-1, 1))
print(actual_predictions)

[[59110.6733943 ]
 [60992.20314916]
 [62821.24154784]
 [64432.56239055]
 [65987.16991583]
 [67566.32246571]
 [69078.71716059]
 [70428.41205017]
 [71648.98952994]
 [72837.18846662]
 [73728.96163769]
 [74166.40090957]]


In [34]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop

# 数据导入
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# 数据预处理
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

# 直接使用keras.Sequential()搭建全连接网络模型
model = Sequential()
model.add(Dense(128, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

#lr为学习率，epsilon防止出现0，rho/decay分别对应公式中的beta_1和beta_2
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.00001)
model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])
print("---------------training--------------")
model.fit(x_train, y_train, epochs=10, batch_size=32)
print('\n')
print("--------------testing----------------")
loss, accuracy = model.evaluate(x_test, y_test)
print('loss:', loss)
print('accuracy:', accuracy)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
---------------training--------------
Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2555 - accuracy: 0.9270
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1213 - accuracy: 0.9644
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0895 - accuracy: 0.9744
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0711 - accuracy: 0.9799
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0598 - accuracy: 0.9832
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0518 - accuracy: 0.9861
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0443 - accuracy: 0.9877
Epoch 8/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0406 - accuracy: 0.9890
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0359 - accuracy: 0

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 模拟数据
X = all_data[:-1]  # 输入序列  
y = all_data[1:]  # 目标序列
# 重塑数据以适应LSTM  
X = X.reshape((X.shape[0], 1, 1))
y = y.reshape((y.shape[0], 1))

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))  # 50个LSTM单元  
model.add(Dense(1))  # 输出层，因为我们的任务是回归，所以这里是1个神经元
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)
test_loss = model.evaluate(X, y)  # 假设X_test和y_test是你的测试数据和标签  
print('Test Mean Absolute Error:', test_loss)

Epoch 1/100
46/46 [==============================] - 0s 8ms/step - loss: 45352264.0000 - mse: 45352264.0000 - val_loss: 1091583744.0000 - val_mse: 1091583744.0000
Epoch 2/100
46/46 [==============================] - 0s 5ms/step - loss: 33599612.0000 - mse: 33599612.0000 - val_loss: 786598912.0000 - val_mse: 786598912.0000
Epoch 3/100
46/46 [==============================] - 0s 5ms/step - loss: 22319968.0000 - mse: 22319968.0000 - val_loss: 467472384.0000 - val_mse: 467472384.0000
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 11963225.0000 - mse: 11963225.0000 - val_loss: 217797152.0000 - val_mse: 217797152.0000
Epoch 5/100
46/46 [==============================] - 0s 6ms/step - loss: 4881410.0000 - mse: 4881410.0000 - val_loss: 72406056.0000 - val_mse: 72406056.0000
Epoch 6/100
46/46 [==============================] - 0s 5ms/step - loss: 1462494.0000 - mse: 1462494.0000 - val_loss: 17912168.0000 - val_mse: 17912168.0000
Epoch 7/100
46/46 [=====================

In [36]:
X.shape

(1825,)